In [37]:
import numpy as np

train_emb = np.load('train_emb.npy')
val_emb = np.load('val_emb.npy')
test_emb = np.load('test_emb.npy')
train_labels = np.load('train_labels.npy')
val_labels = np.load('val_labels.npy')
test_labels = np.load('test_labels.npy')
train_features = np.load('train_features.npy')
val_features = np.load('val_features.npy')
test_features = np.load('test_features.npy')

print(train_emb.shape)
print(val_emb.shape)
print(test_emb.shape)
print(train_labels.shape)
print(val_labels.shape)
print(test_labels.shape)
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

print(train_emb[0])


(90941, 40)
(29799, 40)
(48603, 40)
(90941,)
(29799,)
(48603,)
(90941, 128)
(29799, 128)
(48603, 128)
[-0.139176   -0.24919727  0.25901437 -0.02747912  0.04926306 -0.01338017
 -0.10090951 -0.23536047 -0.26964012  0.05506337  0.24175395 -0.14814366
 -0.43139833 -0.08389322 -0.03078499 -0.18383165 -0.10181541 -0.31361264
 -0.19075754 -0.17027982  0.01980056 -0.13180263  0.14787617  0.17163722
 -0.05474645 -0.1679169  -0.03034001 -0.08303773 -0.13428286 -0.04746383
  0.12286613 -0.05885872 -0.11206771  0.07148973  0.02219984 -0.20059946
  0.04150229  0.09666739  0.03941768  0.08011841]


In [38]:
from hdc.bsc import BSC as HDC
from hdc.hv import HDCRepresentation
from hdc.itemmem import ItemMem
from hdc.encoder import Encoder
from hdc.levelencoder import LevelEncoder
from tqdm import tqdm
from typing import Type

In [39]:
N = 10240
encoder = LevelEncoder(HDC, N, -1, 1, 0.005)

def encode_instance(emb):
    feature_hvs = []
    for i in range(emb.shape[0]):
        feature_hvs.append(encoder.encode(emb[i]))
    return HDC.sequence(feature_hvs)
    

num_intervals: 400
num_bits: 25


In [40]:
train = np.concatenate((train_emb, val_emb), axis=0)
train_original = np.concatenate((train_features, val_features), axis=0)
train_labels = np.concatenate((train_labels, val_labels), axis=0)
print(train.shape)
print(train_labels.shape)

(120740, 40)
(120740,)


In [41]:
def learn(train, train_original, train_labels):
    itemmem = ItemMem(HDC)
    for i in tqdm(range(train.shape[0])):
        instance = train[i]
        label = train_labels[i].astype(int)
        # hv1 = encode_instance(instance)
        # hv2 = encode_instance(train_original[i])
        # hv = HDC.sequence([hv1, hv2])
        hv = encode_instance(instance)
        itemmem.cache(label, hv)
    itemmem.build()
    return itemmem

itemmem = learn(train, train_original, train_labels)

100%|██████████| 120740/120740 [01:11<00:00, 1691.81it/s]


In [42]:
def test(test, test_features, test_labels):
    correct = 0
    for i in tqdm(range(test.shape[0])):
        instance = test[i]
        label = test_labels[i].astype(int)
        # hv1 = encode_instance(instance)
        # hv2 = encode_instance(test_features[i])
        # hv = HDC.sequence([hv1, hv2])
        hv = encode_instance(instance)
        pred = itemmem.query(hv)
        if pred == label:
            correct += 1
    return correct / test.shape[0]

In [43]:
acc = test(test_emb, test_features, test_labels)

100%|██████████| 48603/48603 [01:24<00:00, 577.77it/s]


In [44]:
print(acc)

0.20317675863629817
